# Data Cleaner
Clean data to remove any nan values or large anomalies. 

In [1]:
import os
import xarray as xr
import numpy as np
from tqdm import tqdm

In [2]:
input_dir = 'time_series_losangeles'
output_dir = 'time_series_cleaned_losangeles'
spike_threshold = 2000 #Anything above this is likely an anomaly. 
os.makedirs(output_dir, exist_ok=True)



In [3]:
def DataCleaner(input_dir, output_dir, spike_threshold):
    bands = [f"C{b:02}" for b in range(1, 17)]
    
    for band in bands:
        path = os.path.join(input_dir, f"{band}_time_series.nc")
        if not os.path.exists(path):
            continue
    
        print(f"Cleaning {band}")
        ds = xr.open_dataset(path)
        if "Rad" not in ds:
            print(f"No 'Rad' in {band}, skipping.")
            continue
    
        rad = ds["Rad"]
        rad_data = rad.values.copy()  # shape: (T, Y, X)
    
        # Drop pixels that are always NaN 
        nan_mask = np.all(np.isnan(rad_data), axis=0) # (Y, X)
        valid_y = ~np.all(nan_mask, axis=1)
        valid_x = ~np.all(nan_mask, axis=0)
    
        # Subset spatially
        rad_data = rad_data[:, valid_y, :][:, :, valid_x] #(T, new_Y, new_X)
    

        T, H, W = rad_data.shape #Remove any anomalous spikes 
        spike_mask = rad_data > spike_threshold
        spike_count = 0
    
        for t in tqdm(range(1, T), desc=f'{band} - Despiking'):
            spikes = spike_mask[t]
            spike_count += np.sum(spikes)
            rad_data[t][spikes] = rad_data[t - 1][spikes]  # replace with previous timestep
    
        # Replace into dataset
        ds_clean = ds.isel(x=np.where(valid_x)[0], y=np.where(valid_y)[0])
        ds_clean["Rad"].values = rad_data  # overwrite cleaned data
    
        # Save
        out_path = os.path.join(output_dir, f"{band}_time_series.nc")
        ds_clean.to_netcdf(out_path)
        print(f"Saved cleaned {band} to {out_path}.{spike_count} spikes replaced.")
        print(f"{spike_count} spikes replaced.")

    return

In [4]:
DataCleaner(input_dir, output_dir, spike_threshold)

Cleaning C01


C01 - Despiking: 100%|█████████████████| 14326/14326 [00:01<00:00, 11006.03it/s]


Saved cleaned C01 to time_series_cleaned_losangeles/C01_time_series.nc.0 spikes replaced.
0 spikes replaced.
Cleaning C02


C02 - Despiking: 100%|██████████████████| 14323/14323 [00:05<00:00, 2613.20it/s]


Saved cleaned C02 to time_series_cleaned_losangeles/C02_time_series.nc.0 spikes replaced.
0 spikes replaced.
Cleaning C03


...

Saved cleaned C14 to time_series_cleaned_losangeles/C14_time_series.nc.0 spikes replaced.
0 spikes replaced.
Cleaning C15


C15 - Despiking: 100%|████████████████| 14303/14303 [00:00<00:00, 113511.86it/s]


Saved cleaned C15 to time_series_cleaned_losangeles/C15_time_series.nc.0 spikes replaced.
0 spikes replaced.
Cleaning C16


C16 - Despiking: 100%|████████████████| 14296/14296 [00:00<00:00, 100734.94it/s]


Saved cleaned C16 to time_series_cleaned_losangeles/C16_time_series.nc.0 spikes replaced.
0 spikes replaced.
